### Import libraries
Import libraries, set parameters, and import classes

In [1]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [2]:
from __lib__.NN_class import NeuralNetwork
from __lib__.DataProcessing import PreProcessing

### Neural Networks

Import trained model by specifying the file (filename) of the saved weights and the architecture.
The function import_compile(filename,input_shape) performs a check of the defined parameters wrt the saved model

In [3]:
# file where the model was saved
filename = './results/NN_Elasticity'

# build network, import weights, and compile to make predictions
umat_dim = 4 # dimensional space
material='elastic'
hidden_NN_Stress = 6 # number of hidden layers in NN Evolution

Nnet = NeuralNetwork(umat_dim,
                     material,
                     hidden_NN_Stress,
                     activation_NN_Stress='tanh') # Initialize TANN class

# build TF model class
input_shape = [None,umat_dim]

Nnet.import_compile(filename,input_shape)


Metal device set to: Apple M1 Pro
File ./results/NN_Elasticity refers to the following model:
umat_dim: 4 
material: elastic 
hidden_NN_Stress: 6 
activation_NN_Stress: tanh 
hidden_NN_Evolution: 0 
activation_NN_Evolution: 


2022-05-09 23:16:31.454582: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-09 23:16:31.454758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


#### Inference: loading path

Make predictions of the constitutive response under a simple shear loading path using the trained model. The loading path is predefined (see below), but you have to implement a function that call the trained model and make predictions.

Note. All quantities should be physical (not normalized)

In [4]:
increments = 30
sub_increments = increments//2
de = 1e-3
umat_dir = -1

dstrain = np.zeros((increments,umat_dim))
dstrain[:sub_increments,umat_dir] = de*np.ones((sub_increments))
dstrain[sub_increments:2*sub_increments,umat_dir] = -de*np.ones((sub_increments))

strain = np.zeros((1,umat_dim))

#### 1. Make predictions

Here goes your function to predict the stress and internal variables deriving from the deformation path specified by dstrain

In [5]:
hstress = ???

SyntaxError: invalid syntax (3475195124.py, line 1)

#### Import reference model solution

In [6]:
file = './data/3D_PlaneStrain_Elasticity_CSMA_inference'
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain_ref,svars_ref,stress_ref,energy_ref,dissipationDot_ref = data

#### (+) Compute elastic energy

In [7]:
def computeEnergy_elasticity(strain,stress):
    energy = 0.5*np.einsum('ij,ij->i',strain,stress,order='F')
    return energy

henergy = computeEnergy_elasticity(hstrain,hstress)

NameError: name 'hstrain' is not defined

#### Compare results

Plot the stress vs strain history as predicted by the model and compare it with the one of the reference model

In [8]:
plt.plot(hstrain[:,umat_dir],
         hstress[:,umat_dir],
         color = 'darkblue',
         alpha=1,linewidth=1, marker='o',markersize=5,label='NN')
plt.plot(strain_ref[:,umat_dir],
         stress_ref[:,umat_dir],
         color='darkorange',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\sigma= NN(\\varepsilon)$")
plt.legend()
plt.show()

NameError: name 'hstrain' is not defined

Plot the free energy vs strain history as predicted by the model and compare it with the one of the reference model

In [ ]:
plt.plot(hstrain[:,umat_dir],
         henergy,
         color = 'darkblue',
         alpha=1,linewidth=1, marker='o',markersize=5,label='NN')

plt.plot(strain_ref[:,umat_dir],
         energy_ref,
         color = 'coral',
         alpha=1,linewidth=2, marker='s',markersize=0,label='ref')
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\psi$")
plt.legend()
plt.show()

#### Compute and plot the error with respect to the reference constitutive model

In [ ]:
MAE_energy = (np.abs(energy_ref-henergy))
MAE_stress = (np.abs(stress_ref[:,umat_dir]-hstress[:,umat_dir]))

In [ ]:
plt.plot(hstrain[:,umat_dir],
         MAE_energy,
         alpha=1,linewidth=1, marker='o',markersize=4,label='$MAE_{\psi}$')

plt.plot(hstrain[:,umat_dir],
         MAE_stress,
          alpha=1,linewidth=1, marker='s',markersize=4,label='$MAE_{\sigma}$')
plt.xlabel("$\\varepsilon$")
plt.ylabel("Mean Absolute Error")
plt.legend()
plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>